In [ ]:
!pip install mplcyberpunk

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import matplotlib.pyplot as plt
import mplcyberpunk
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SQLContext
from pyspark.ml.feature import StandardScaler
from sklearn.preprocessing import StandardScaler
from pyspark.sql import SparkSession
import numpy as np

In [3]:
spark= SparkSession.builder.appName("Cartofior").getOrCreate()

In [ ]:

mydataset = spark.read.csv('C:/Users/catal/OneDrive/Desktop/hotel_bookings.csv',inferSchema=True,
                     header=True)

In [9]:
mydataset.printSchema()


root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

In [10]:
mydataset.describe().show()

+-------+------------+-------------------+------------------+------------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------------------+-------------------+--------------------+---------+-------+--------------+--------------------+-------------------+----------------------+------------------------------+------------------+------------------+-------------------+------------+-----------------+------------------+--------------------+---------------+------------------+---------------------------+-------------------------+------------------+
|summary|       hotel|        is_canceled|         lead_time| arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|            adults|           children|              babies|     meal|country|market_segment|distribution_channel|  is_repeated_guest|previous_cancellations|previous_bookings_not_cance

In [11]:
mydataset.columns

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [126]:
assembler = VectorAssembler(inputCols=['lead_time','total_of_special_requests','previous_cancellations','is_repeated_guest','stays_in_weekend_nights',
 'stays_in_week_nights','booking_changes','previous_bookings_not_canceled'],outputCol='features')

In [127]:
output=assembler.transform(mydataset)

In [128]:
final_data=output.select('features','is_canceled')

In [129]:
from pyspark.ml.classification import LogisticRegression

In [130]:
train_adults,test_coloane=final_data.randomSplit([0.7,0.3])
lr_adults=LogisticRegression(labelCol='is_canceled')

In [131]:
fitted_model=lr_adults.fit(train_adults)

In [132]:
training_sum=fitted_model.summary

In [133]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|        is_canceled|         prediction|
+-------+-------------------+-------------------+
|  count|              83721|              83721|
|   mean|0.37172274578660075| 0.2111417684929707|
| stddev|0.48326776806145477|0.40812119718340645|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [134]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [135]:
pred_and_labels=fitted_model.evaluate(test_coloane)

In [136]:
pred_and_labels.predictions.show()

+--------------+-----------+--------------------+--------------------+----------+
|      features|is_canceled|       rawPrediction|         probability|prediction|
+--------------+-----------+--------------------+--------------------+----------+
|     (8,[],[])|          0|[0.71310777961390...|[0.67108750147998...|       0.0|
|     (8,[],[])|          0|[0.71310777961390...|[0.67108750147998...|       0.0|
|     (8,[],[])|          1|[0.71310777961390...|[0.67108750147998...|       0.0|
|     (8,[],[])|          1|[0.71310777961390...|[0.67108750147998...|       0.0|
| (8,[0],[1.0])|          0|[0.70819454636911...|[0.67000209789172...|       0.0|
| (8,[0],[2.0])|          0|[0.70328131312432...|[0.66891487960305...|       0.0|
| (8,[0],[4.0])|          0|[0.69345484663474...|[0.66673503339928...|       0.0|
| (8,[0],[6.0])|          1|[0.68362838014515...|[0.66454803254003...|       0.0|
| (8,[0],[9.0])|          0|[0.66888868041078...|[0.66125427150987...|       0.0|
|(8,[0],[17.0])|

In [137]:
canceled_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='is_canceled')

In [138]:
auc = canceled_eval.evaluate(pred_and_labels.predictions)

In [139]:
auc

0.6497742350473751